In [ ]:
import os
import obspy
from obspy import read
import numpy as np
import matplotlib.pyplot as plt
from obspy.imaging.spectrogram import spectrogram
from obspy.signal.trigger import classic_sta_lta
DATA_DIR = '/app/data'


In [ ]:
def read_seismic_files(directory):
    st = obspy.Stream()
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file matches the expected naming pattern
            # At least 5 parts in the filename
            if len(file.split('.')) >= 5:  
                file_path = os.path.join(root, file)
                try:
                    st += read(file_path)
                except Exception as e:
                    print(f"Error reading file {file}: {str(e)}")
    return st

In [ ]:
def display_information(stream):
    stream = read_seismic_files(DATA_DIR)
    
    print(f"Number of traces: {len(stream)}")
    station_ids = set(tr.stats.station for tr in stream)
    print("Unique station IDs in the dataset:")
    for station_id in sorted(station_ids):
        print(station_id)
    
    for i, tr in enumerate(stream):
        print(f"\nTrace {i + 1}:")
        print(f"Station: {tr.stats.station}")
        print(f"Channel: {tr.stats.channel}")
        print(f"Start time: {tr.stats.starttime}")
        print(f"End time: {tr.stats.endtime}")
        print(f"Number of samples: {tr.stats.npts}")
        print(f"Sampling rate: {tr.stats.sampling_rate} Hz")
display_information(stream)


In [ ]:
def plat_single_trace(stream):
    stream[0].plot(type='relative', outfile=None, fig=plt.figure(figsize=(12, 6)))

    # Adjust the title and grid after ObsPy has created the plot
    plt.title(f"Seismic Data: {stream[0].stats.station} - {stream[0].stats.channel}")
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)

    plt.show()
plat_single_trace(stream)

In [ ]:
def plot_multiple_traces(stream):
    fig, axes = plt.subplots(5, 1, figsize=(12, 15), sharex=True)
    fig.suptitle("Seismic Data: Multiple Traces", fontsize=16)
     # Plot first 5 traces
    for i, tr in enumerate(stream[:5]): 
        ax = axes[i]
        times = tr.times()
        ax.plot(times, tr.data, 'k')
        ax.set_title(f"{tr.id}", fontsize=10)
        ax.set_ylabel("Amplitude (counts)")
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        
        # Set y-axis limits symmetrically
        max_amp = max(abs(tr.data.max()), abs(tr.data.min()))
        ax.set_ylim(-max_amp, max_amp)
        
        # Only show x-axis label for the bottom subplot
        if i == 4:
            ax.set_xlabel("Time (seconds)")
        
        # Adjust y-axis tick label font size
        ax.tick_params(axis='y', labelsize=8)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)  
    plt.show()
plot_multiple_traces(stream)

In [ ]:
def plot_timeseries(stream, station=None, channel=None):
    if station and channel:
        st_filtered = stream.select(station=station, channel=channel)
    else:
        st_filtered = stream

    fig, ax = plt.subplots(figsize=(12, 6))
    # For each trace in the filtered stream, it plots the data against time.
    for tr in st_filtered:
        times = tr.times()
        ax.plot(times, tr.data, label=f'{tr.stats.station}.{tr.stats.channel}')
    
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Amplitude')
    ax.set_title('Seismic Timeseries')
    ax.legend()
    ax.grid(True)
    plt.show()

plot_timeseries(stream)